## Домашнее задание к лекции «Основы веб-скрапинга»

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

#### Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту <a href="https://habr.com/ru/search/">*habr.com*</a>. 
Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

**<дата> - <заголовок> - <ссылка на материал>**

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. 

Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [2]:
req_list = ['python','анализ данных']

In [34]:
def get_habr_posts(req_list):
    url = 'https://habr.com/ru/search/'
    habr_posts = pd.DataFrame()
    for req_text  in req_list:
        params = {
            'q' : req_text
        }
        posts = BeautifulSoup(requests.get(url, params).text).find_all('article', class_='tm-articles-list__item')
        for article in posts:
            date = pd.to_datetime(article.find('time').get('datetime'), format = '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')
            if article.find('h2','tm-title tm-title_h2'):
                title = article.find('h2','tm-title tm-title_h2').text.strip()
                link = url[:-11] + article.find('h2','tm-title tm-title_h2').find('a').get('href')
            else:
                title = article.find('h2','tm-megapost-snippet__title').text.strip()
                link = url[:-11] + article.find('div','tm-megapost-snippet__tint').find('a').get('href')
            row = {'date': date, 'title': title, 'link': link}
            habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])
    return habr_posts.reset_index(drop=True)

In [6]:
habr_posts = get_habr_posts(req_list).drop_duplicates(keep='first')
habr_posts.head()

,date,title,link
0,2020-03-03,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...
1,2023-06-23,Как можно компилировать типизированный Python,https://habr.com/ru/companies/piter/articles/7...
2,2023-06-20,"Типизация в Python. Работа с Mypy, PyCharm и S...",https://habr.com/ru/companies/selectel/article...
3,2023-05-31,25 бесплатных курсов по Python 2023 года,https://habr.com/ru/articles/738438/
4,2019-10-31,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/473926/


#### Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

**<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>**

In [41]:
def get_habr_posts_add(req_list,pages):
    
    habr_posts_add = pd.DataFrame()
    for req_text  in req_list:
        params = {
            'q' : req_text
        }

        for i in range(1, pages+1):
            params['page'] = i
            url = 'https://habr.com/ru/search/page' + str(i)+'/'
            posts = BeautifulSoup(requests.get(url, params).text).find_all('article', class_='tm-articles-list__item')
            for article in posts:
                date = pd.to_datetime(article.find('time').get('datetime'), format = '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')
                if article.find('h2','tm-title tm-title_h2'):
                    title = article.find('h2','tm-title tm-title_h2').text
                    link = url[:-17] + article.find('h2','tm-title tm-title_h2').find('a').get('href')
                else:
                    title = article.find('h2','tm-megapost-snippet__title').text
                    link = url[:-17] + article.find('div','tm-megapost-snippet__tint').find('a').get('href')
                post = BeautifulSoup(requests.get(link).text).find('div','article-formatted-body article-formatted-body article-formatted-body_version-1')
                if post:
                    if post.find('div','spoiler'):
                        post.find('div','spoiler').extract()
                elif BeautifulSoup(requests.get(link).text).find('div','tm-article-body'):
                    post = BeautifulSoup(requests.get(link).text).find('div','tm-article-body') 
                else:
                    post = BeautifulSoup(requests.get(link).text).find('div','tm-error-message__body')
                text = post.text.strip()
                if BeautifulSoup(requests.get(link).text).find('div','tm-error-message__body'):
                    likes = None
                else:
                    likes = BeautifulSoup(requests.get(link).text)
                    #если суммарный рейтинг положительный
                    if likes.find('div', 'tm-votes-meter tm-article-rating__votes-switcher').find('span','tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                        likes = int(likes.find('div', 'tm-votes-meter tm-article-rating__votes-switcher').find('span','tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text[1:])
                    #если суммарный рейтинг отрицательный
                    elif likes.find('div', 'tm-votes-meter tm-article-rating__votes-switcher').find('span','tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                        likes = int(likes.find('div', 'tm-votes-meter tm-article-rating__votes-switcher').find('span','tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text)         
                    #если суммарный рейтинг нейтральный (нулевой)
                    else:
                        likes = int(BeautifulSoup(requests.get(link).text).find('div', 'tm-votes-meter tm-article-rating__votes-switcher').find('span','tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text)
                row = {'date': date, 'title': title, 'link': link, 'text': text, 'likes':likes}
                habr_posts_add = pd.concat([habr_posts_add, pd.DataFrame([row])])
    return habr_posts_add.reset_index(drop=True)

In [35]:
habr_posts_add = get_habr_posts_add(req_list,1).drop_duplicates(keep='first')

In [40]:
habr_posts_add.head()

,date,title,link,text,likes
0,2020-03-03,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...,"Согласно отчету RedMonk за январь 2020 года, P...",31
1,2023-06-23,Как можно компилировать типизированный Python,https://habr.com/ru/companies/piter/articles/7...,"Прошло уже целых 9 лет с тех пор, как состоялс...",11
2,2023-06-20,"Типизация в Python. Работа с Mypy, PyCharm и S...",https://habr.com/ru/companies/selectel/article...,"Привет, Хабр! Мы в Selectel много программируе...",12
3,2023-05-31,25 бесплатных курсов по Python 2023 года,https://habr.com/ru/articles/738438/,"Тема обучения «Питону» — вечнозелёная, посколь...",7
4,2019-10-31,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/473926/,Создатель языка программирования Python Гвидо ...,23
